# 03 - Interactive Backtest Runner

This notebook is used to interactively run, debug, and analyze the `ZeroDTESpread` strategy.

In [2]:
import backtrader as bt
import pandas as pd
from pathlib import Path
import sys

# Add the project's 'src' directory to the Python path
# This allows us to import our custom modules
src_path = Path('../src').resolve()
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# --- Import our custom classes ---
from strategies.zero_dte_spread import ZeroDTESpread
from data_feeds import OptionsDataPandas

## 1. Load Data

Load the SPX and options data from the Parquet files generated by `create_mock_data.py`.

In [3]:
DATA_DIR = Path("../data/parquet")
spx_file = DATA_DIR / "SPX_20250613.parquet"
options_file = DATA_DIR / "SPX_OPTIONS_20250613.parquet"

# Load SPX Data
spx_df = pd.read_parquet(spx_file)
print("SPX Data Loaded:")
print(spx_df.head())

# Load Options Data
options_df = pd.read_parquet(options_file)
print("\nOptions Data Loaded:")
print(options_df.head())

SPX Data Loaded:
                            open         high          low        close  \
timestamp                                                                 
2025-06-13 09:30:00  5200.566429  5200.566429  5200.566429  5200.566429   
2025-06-13 09:31:00  5200.810759  5200.810759  5200.810759  5200.810759   
2025-06-13 09:32:00  5200.672722  5200.672722  5200.672722  5200.672722   
2025-06-13 09:33:00  5201.732793  5201.732793  5201.732793  5201.732793   
2025-06-13 09:34:00  5201.621421  5201.621421  5201.621421  5201.621421   

                     volume  
timestamp                    
2025-06-13 09:30:00    2678  
2025-06-13 09:31:00    2831  
2025-06-13 09:32:00    1285  
2025-06-13 09:33:00    1140  
2025-06-13 09:34:00    1725  

Options Data Loaded:
                     underlying_price contract_type  expiration  strike  \
timestamp                                                                 
2025-06-13 09:30:00       5200.566429           put  2025-06-13    5180   


## 2. Configure and Run Backtest

Here, we set up the Cerebro engine, add the strategy, feed the data, and run the backtest.

In [3]:
cerebro = bt.Cerebro()

# --- Add Strategy ---
# You can change parameters here for quick experiments
cerebro.addstrategy(
    ZeroDTESpread,
    delta_target=0.12,
    rsi_overbought=65,
    rsi_oversold=35
)

# --- Add Data Feeds ---
# 1. SPX Data Feed
spx_data_feed = bt.feeds.PandasData(
    dataname=spx_df,
    fromdate=spx_df.index[0],
    todate=spx_df.index[-1]
)
cerebro.adddata(spx_data_feed, name='SPX')

# 2. Options Data Feed (Custom)
dummy_options_clock = pd.DataFrame(index=spx_df.index)
options_data_feed = OptionsDataPandas(
    dataname=dummy_options_clock,
    options_chain=options_df # Pass the full options dataframe here
)
cerebro.adddata(options_data_feed, name='Options')

# --- Configure Cerebro ---
cerebro.broker.setcash(100000.0)
cerebro.addsizer(bt.sizers.FixedSize, stake=10)
cerebro.broker.setcommission(commission=0.65, mult=100)

# --- Run Backtest ---
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
results = cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00


IndexError: .iloc requires numeric indexers, got [[5200.566429343765 'put' datetime.date(2025, 6, 13) ...
  -0.49073019613246843 -0.3029211405091701 -0.1]
 [5200.566429343765 'put' datetime.date(2025, 6, 13) ...
  0.5025100084259962 0.6606107290995588 0.0]
 [5200.566429343765 'put' datetime.date(2025, 6, 13) ...
  1.5690722497243201 1.6862628585619446 0.1]
 ...
 [5200.717145330401 'put' datetime.date(2025, 6, 13) ...
  1.572617303595196 1.6473595290273255 0.1]
 [5200.717145330401 'put' datetime.date(2025, 6, 13) ...
  2.5925005390906555 2.683750413945941 0.2]
 [5200.717145330401 'put' datetime.date(2025, 6, 13) ...
  3.540311257296787 3.668620908660424 0.30000000000000004]]

## 3. Plot Results

Visualize the backtest results. The plot will appear directly in the notebook.

In [ ]:
%matplotlib inline
cerebro.plot(style='candlestick', barup='green', bardown='red', iplot=True)